In [54]:
!git add Parser.ipynb
!git commit -m "init"
!git push

[master c4dcdf5] init
 1 file changed, 878 insertions(+)
 create mode 100644 OpenDataRussia/OrthodoxChurh/Parser.ipynb
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (5/5), 5.60 KiB | 0 bytes/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/DmitrySerg/OpenData
   4b416e4..c4dcdf5  master -> master


## Цель
- Собрать базу открытых данных с сайта Московской патриархии о персоналиях, организациях, богослужебных указаниях и храмах, структурировать их и загрузить в хаб открытых данных

## Задача
- проинвентаризировать данные на сайте Московской патриархии
- написать парсеры и выгрузить данные в форматах CSV и JSON
- загрузить данные в хаб открытых данных http://hubofdata.ru в удобном виде.
## Требования
- открытый исходный код в Github под свободной лицензией
- межплатформенный код (возможность запуска на Linux/Windows)
- предусмотреть перенос всех метаданных в CKAN в виде тегов и атрибутов
- переносить все данные в CKAN или на отдельный хостинг (например, в Github)
## Данные
- Персоналии - http://www.patriarchia.ru/db/persons/
- Организации - http://www.patriarchia.ru/db/organizations/
- Богослужебные указания - http://www.patriarchia.ru/bu/
- Храмы - http://map.patriarhia.ru/
## Пожелания
- реализация в виде программы командной строки или веб-приложения
- возможность запуска на MacOSX
- использовать скриптовые языки такие как Python, Perl, R и другие.

In [133]:
import requests      
import numpy as np   
import pandas as pd  
import time          
from bs4 import BeautifulSoup

import string

In [62]:
main_link = "http://www.patriarchia.ru"
link_persons = "http://www.patriarchia.ru/db/persons/"
link_organizations = "http://www.patriarchia.ru/db/organizations/"
link_methodology = "http://www.patriarchia.ru/bu/"
link_churches = "http://map.patriarhia.ru/"

In [209]:
def getSoup(link):
    """
        Returns bs4-formatted page
    """
    html = requests.get(link)
    if html.status_code == 200:
        soup = BeautifulSoup(html.content, 'lxml')
    else:
        soup = None
    return soup

In [73]:
persons = requests.get(link_persons)

In [5]:
html = persons.content

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [366]:
persons = soup.find('div', attrs={"class":"main"})

In [367]:
persons = persons.findAll('a', href=True)

In [368]:
persons[0:5]

[<a href="/db/text/2147951.html">Августин, епископ Городецкий и Ветлужский (Анисимов Анатолий Иванович)</a>,
 <a href="http://www.patriarchia.ru/db/persons/31911/">Епископат РПЦ</a>,
 <a href="/db/text/52361.html">Августин, митрополит Белоцерковский и Богуславский (Маркевич Адам Иванович)</a>,
 <a href="http://www.patriarchia.ru/db/persons/31911/">Епископат РПЦ</a>,
 <a href="/db/text/424847.html">Агапит, архиепископ Штутгартский (РПЦЗ), викарий Берлинской епархии (Горачек Александр Владимирович)</a>]

In [369]:
persons[0]['href']

'/db/text/2147951.html'

In [383]:
dfPersons = []

for i in range(0, len(persons)-1, 2):
    name = persons[i].text
    link = main_link + persons[i]['href']
    affiliation = persons[i+1].text
    dfPersons.append({"name":name, "link":link, "affiliation":affiliation})

In [387]:
pd.DataFrame(dfPersons).head()

,affiliation,link,name
0,Епископат РПЦ,http://www.patriarchia.ru/db/text/2147951.html,"Августин, епископ Городецкий и Ветлужский (Ани..."
1,Епископат РПЦ,http://www.patriarchia.ru/db/text/52361.html,"Августин, митрополит Белоцерковский и Богуслав..."
2,Епископат РПЦ,http://www.patriarchia.ru/db/text/424847.html,"Агапит, архиепископ Штутгартский (РПЦЗ), викар..."
3,Епископат РПЦ,http://www.patriarchia.ru/db/text/67725.html,"Агапит, митрополит Могилев-Подольский и Шаргор..."
4,Епископат РПЦ,http://www.patriarchia.ru/db/text/3667680.html,"Агафангел, епископ Норильский и Туруханский (Д..."


In [379]:
def getPersonData(link):
    """
        Returns a dict with parsed field:value pairs from personal page
    
        link: string
            link to personal page
    """
    
    # first - get the page content
    pageContent = getSoup(link)
    pageContent = pageContent.find('div', attrs={'class':'main'})
    pageContent = pageContent.findAll("dl")
    
    # next - find all title-text pairs 
    data = {}
    for text in pageContent:
        field_names = text.findAll('dt')
        if field_names == []:
            next
        else:
            field_names = [f.getText().strip() for f in field_names]
            field_values = text.findAll('dd')
            field_values = [f.getText().strip() for f in field_values]
            text = dict(zip(field_names, field_values))
            data.update(text)
            
    return data

In [382]:
from tqdm import tqdm_notebook

In [412]:
final = []

for person in tqdm_notebook(dfPersons):
    try:
        data = getPersonData(person['link'])
        person.update(data)
        final.append(person)
        time.sleep(1)
    except:
        print('problem',  person['link'])
        time.sleep(30)

problem http://www.patriarchia.ruhttp://www.patriarchia.ru/db/persons/30920/
problem http://www.patriarchia.ruhttp://www.patriarchia.ru/db/persons/31911/


KeyboardInterrupt: 

In [413]:
pd.DataFrame(final).to_csv("final.csv")

In [419]:
dfPersons[390]

{'affiliation': 'Панкратий, епископ Троицкий, викарий Святейшего Патриарха Московского и всея Руси (Жердев Владислав Петрович)',
 'link': 'http://www.patriarchia.ruhttp://www.patriarchia.ru/db/persons/31911/',
 'name': 'Епископат РПЦ'}